In [4]:
import numpy_financial as npf
import pandas as pd
import numpy as np

In [2]:
def monthly_PI(interest, years, price, yearly_months=12):
    monthly_PI = -1 * npf.pmt(interest/yearly_months, years*yearly_months, price)
    return monthly_PI

In [5]:
# purchase
purchase_price = 600e3
downpayment = 21e3
rehab_cost = 100e3
after_repair_value = 800e3

# acquisition
yearly_months = 12
acq_yearly_interest = 0.065
mortgage_years = 30
rehab_months = 6
monthly_taxes = 555 # from zillow

# initial rental period
monthly_rent = 4e3
vacancy_frac, repairs_frac, capex_frac = 0.05, 0.05, 0.05

# refinanced rental period 
ref_yearly_interest = 0.065
refinance_months = 9
pre_refi_duration = refinance_months-rehab_months

In [7]:
class Mortgage():
    def __init__(self, yearly_interest, loan_amount, total_years=30):
        self.yearly_interest = yearly_interest
        self.total_years = total_years
        self.loan_amount = loan_amount
        
        # self.yearly_interest /= 100
        self.monthly_interest = self.yearly_interest / 12
        self.num_payments = total_years * 12
        self.monthly_PI = self.monthly_PI()
        self.df = self.amortization_df()
    
    def monthly_PI(self):
        monthly_PI = self.loan_amount * (self.monthly_interest * np.power(1 + self.monthly_interest, self.num_payments)) / (np.power(1 + self.monthly_interest, self.num_payments) - 1)
        return monthly_PI
    
    def amortization_df(self):

        # # Compute monthly payment
        # monthly_payment = principal * (monthly_interest * np.power(1 + monthly_interest, self.num_payments)) / (np.power(1 + monthly_interest, self.num_payments) - 1)
        
        # Create an array for the number of payments
        payment_num = np.arange(1, self.num_payments + 1)

        # Calculate cumulative interest factor
        interest_factor = np.power(1 + self.monthly_interest, payment_num)
        
        # Compute the remaining balance for each payment
        remaining_balance = self.loan_amount * interest_factor - (self.monthly_PI / self.monthly_interest) * (interest_factor - 1)
        
        # Compute the interest for each payment
        interest_payment = remaining_balance * self.monthly_interest
        
        # Subtract to get the principal for each payment
        principal_payment = self.monthly_PI - interest_payment

        # Put these arrays into a DataFrame
        data = {
            'Payment': np.full(self.num_payments, self.monthly_PI),
            'Principal': principal_payment,
            'Interest': interest_payment,
            'Remaining Balance': remaining_balance
        }
        amortization_schedule = pd.DataFrame(data)
        return amortization_schedule

# mort = Mortgage(0.065, 600e3, 30)
# mort.monthly_PI, mort.monthly_interest, mort.num_payments, mort.df

In [14]:
refi.__dict__, acq.price['monthly_PI'], str(acq)

({'time': {'total_months': 9},
  'price': {'monthly_rent': 4000.0,
   'home_value': 800000.0,
   'vacancy_frac': 0.05,
   'repairs_frac': 0.05,
   'capex_frac': 0.05,
   'monthly_PI': 4095.8007922344154,
   'monthly_taxes': 555,
   'monthly_vacancy': 200.0,
   'monthly_repairs': 200.0,
   'monthly_capex': 200.0,
   'loan_fees': 8000.0,
   'mortgage': 648000.0,
   'monthly_OpEx': 1155.0,
   'monthly_expenses': 5250.800792234415,
   'monthly_cashflow': -1250.800792234415,
   'NOI': 34140.00000000001},
  'exponent': {'yearly_interest': 0.065},
  'mort': <__main__.Mortgage at 0x7f86641bddd8>},
 3697.597937433847,
 '\nAcquisition:\nDown Payment: $21000.0\nLoan Amount: $585000.0\nLoan Points/Fees: $6000.0\nLoan Interest Rate: 6.5%\nMonthly P&I: $3697.60')

In [15]:
class Acquisition():
    """ Could inherent a Mortgage object if it could contain several methods """
    def __init__(self, purchase_price, downpayment, yearly_interest):
        self.time = {}
        self.price = {
            'home_value': purchase_price,
            'downpayment': downpayment,
            'mortgage': np.nan,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes,
            
        } 
        self.exponent ={
            'yearly_interest': yearly_interest,
        }
        self.mort = None
        
        self.derive_properties()
        
    def derive_properties(self):
#         super().__init__(self.price['yearly_interest'], 30, self.price['price'])
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = self.price['home_value'] - self.price['downpayment'] + self.price['loan_fees']
        self.price['closing'] = self.price['home_value'] * 0.01 

        self.mort = Mortgage(self.exponent['yearly_interest'], self.price['mortgage'])
        self.price['monthly_PI'] = self.mort.monthly_PI

    def __str__(self):
        return (
            '\nAcquisition:\n'
            f"Down Payment: ${acq.price['downpayment']}\n"
            f"Loan Amount: ${acq.price['mortgage']}\n"
            f"Loan Points/Fees: ${acq.price['loan_fees']}\n"
            f"Loan Interest Rate: {acq.exponent['yearly_interest']*100}%\n"
            f"Monthly P&I: ${acq.price['monthly_PI']:.2f}"
         )
        
class Rehab():
    def __init__(self, rehab_months, total_cost, monthly_PI, monthly_insurance=0, other_costs = 0):
        self.time = {
            'total_months' : rehab_months
        }
        self.price= {
            'total_cost': total_cost,
            'monthly_insurance': monthly_insurance,
            'other': other_costs,
            'holding_cost': np.nan,
            'monthly_PI': monthly_PI,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
        }

        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_rehab'] = self.price['total_cost']*self.time['total_months']/yearly_months
        self.price['holding_cost'] = (self.price['monthly_PI'] + self.price['monthly_taxes']) * self.time['total_months']
        self.price['monthly_total'] = self.price['monthly_rehab'] + self.price['monthly_taxes'] + self.price['monthly_PI']
        
    def __str__(self):
        return (
            '\nRehab:\n'
            f"Holding costs: ${self.price['holding_cost']:.2f}\n"
            f"Monthly P&I: ${self.price['monthly_PI']:.2f}\n"
            f"Rehab time: {self.time['total_months']} months\n"
            f"Monthly taxes: ${self.price['monthly_taxes']}"
        )
    
class PreReFi_Rent():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, vacancy_frac, repairs_frac, capex_frac, total_time, 
                monthly_PI):
        self.time = {
            'total_months': total_time
        }
        self.price =  {
            'monthly_rent': monthly_rent,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_taxes': monthly_taxes,
            'monthly_PI': monthly_PI
        } 
        self.exponent = {}
        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['monthly_OpEx'] = self.sum_opex()
        self.price['monthly_expenses'] = self.price['monthly_OpEx'] + self.price['monthly_PI']
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_opex(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] #+ self.price['monthly_PI']
        return exp_sum
    
    def __str__(self):
        return (
            '\nInitial Rental Period:\n'
            f"Monthly Income: ${self.price['monthly_rent']}\n"
            f"Monthly Expenses: ${self.price['monthly_expenses']:.2f}\n"
            f"Monthly Cashflow: ${self.price['monthly_cashflow']:.2f}\n"
            f"NOI: ${self.price['NOI']}"
        )

class Refinance():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, home_value, vacancy_frac, repairs_frac, capex_frac, refinance_months, yearly_interest):
        self.time =  {
            'total_months': refinance_months
        }
        self.price =  {
            'monthly_rent': monthly_rent,
            'home_value': home_value,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
            'yearly_interest': yearly_interest #0#0.065
        }
        self.mort = None
        self.derive_properties()

    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = 0.8 * self.price['home_value'] + self.price['loan_fees']
        
        self.mort = Mortgage(self.exponent['yearly_interest'], self.price['mortgage'])
        self.price['monthly_PI'] = self.mort.monthly_PI
        # self.price['monthly_PI'] = monthly_PI(self.exponent['yearly_interest'], 30, self.price['mortgage'])
        self.price['monthly_OpEx'] = self.sum_opex()
        self.price['monthly_expenses'] = self.price['monthly_OpEx'] + self.price['monthly_PI']
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_opex(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] #+ self.price['monthly_PI']
        return exp_sum
    
    def __str__(self):
        return (
            '\nRefinance:\n'
            f"Loan Amount: ${refi.price['mortgage']}\n"
            f"Loan Points/Fees: ${refi.price['loan_fees']}\n"
            f"Loan Interest Rate: {refi.exponent['yearly_interest']*100}%\n"
            f"Monthly P&I: ${refi.price['monthly_PI']:.2f}"
        )

In [16]:
acq = Acquisition(purchase_price, downpayment, acq_yearly_interest)
rehab = Rehab(rehab_months, rehab_cost, acq.price['monthly_PI'])
pre_refi = PreReFi_Rent(monthly_rent, vacancy_frac, repairs_frac, capex_frac, 
                        pre_refi_duration, acq.price['monthly_PI'])
refi = Refinance(monthly_rent, after_repair_value, vacancy_frac, repairs_frac, capex_frac, refinance_months, ref_yearly_interest)

In [17]:
print(str(acq))
print(str(rehab))
print(str(pre_refi))
print(str(refi))


Acquisition:
Down Payment: $21000.0
Loan Amount: $585000.0
Loan Points/Fees: $6000.0
Loan Interest Rate: 6.5%
Monthly P&I: $3697.60

Rehab:
Holding costs: $25515.59
Monthly P&I: $3697.60
Rehab time: 6 months
Monthly taxes: $555

Initial Rental Period:
Monthly Income: $4000.0
Monthly Expenses: $4852.60
Monthly Cashflow: $-852.60
NOI: $34139.99999999999

Refinance:
Loan Amount: $648000.0
Loan Points/Fees: $8000.0
Loan Interest Rate: 6.5%
Monthly P&I: $4095.80


In [18]:
cash_required = acq.price['downpayment'] + rehab.price['total_cost'] + acq.price['closing']
cash_outlay = cash_required + rehab.price['holding_cost']
acq_cash_on_cash_roi = pre_refi.price['monthly_cashflow']*yearly_months/cash_required
ref_cash_on_cash_roi = refi.price['monthly_cashflow']*yearly_months/cash_required

pro_forma_cap_rate = pre_refi.price['NOI']/refi.price['mortgage']
purchase_cap_rate = pre_refi.price['NOI']/acq.price['mortgage']

In [90]:
import pandas as pd
import numpy as np

class YearlySummary:
    def __init__(self, acq, rehab, pre_refi, refi, total_years):
        self.acq = acq
        self.rehab = rehab
        self.pre_refi = pre_refi
        self.refi = refi
        self.total_years = total_years
        self.cash_required = self.acq.price['downpayment'] + self.rehab.price['total_cost'] + self.acq.price['closing']
        
        self.rehab_months = self.applicable_months_per_year(self.rehab.time['total_months'], total_years)
        self.rental_months = 12 - self.rehab_months
        self.acq_months = self.applicable_months_per_year(self.refi.time['total_months'], total_years)
        self.refi_months = 12 - self.acq_months
        self.pre_refi_months = self.applicable_months_per_year(self.pre_refi.time['total_months'], total_years) #- self.rehab_months

    def calculate_annual_data(self):
        data = []
        for year in range(0, self.total_years):
            annual_data = {}
            annual_data['Year'] = year
            annual_data['Month'] = year * yearly_months
            annual_data['renting months'] = self.rental_months[year]
            annual_data['Total Annual Income'] = self.pre_refi.price['monthly_rent'] * annual_data['renting months']
            
            annual_data['Operating Expenses'] = self.pre_refi.price['monthly_OpEx'] * self.pre_refi_months[year] + self.refi.price['monthly_OpEx'] * self.refi_months[year]
            annual_data['Mortgage Payment'] = self.acq.price['monthly_PI'] * self.acq_months[year] + self.refi.price['monthly_PI'] * self.refi_months[year]
            annual_data['Total Annual Expenses'] = annual_data['Operating Expenses'] + annual_data['Mortgage Payment']
            
            annual_data['Total Annual Cashflow'] = annual_data['Total Annual Income'] - annual_data['Total Annual Expenses']
            annual_data['Cash on Cash ROI'] = annual_data['Total Annual Cashflow'] / self.cash_required
            annual_data['Property Value'] = (self.acq.price['home_value'] + self.rehab.price['total_cost'])*self.acq_months[year]/12 + self.refi.price['home_value']*self.refi_months[year]/12
            
            annual_data['Loan Balance'] = self.acq.mort.df.iloc[year]['Remaining Balance']  # this is a simplification, in reality you would need to calculate this based on the amortization schedule of the loan
            annual_data['Equity'] = annual_data['Property Value'] - annual_data['Loan Balance']
            # annual_data['Cummulative Profit'] = annual_data['Equity'] + annual_data['Total Annual Cashflow']
            
            if year == 0:
                equity_gain = annual_data['Equity']
            else:
                equity_gain = annual_data['Equity'] - data[year-1]['Equity']
            
            annual_data['Equity Gain'] = equity_gain
            annual_data['Annual Profit'] = annual_data['Equity Gain'] + annual_data['Total Annual Cashflow']
            annual_data['Annualized Total Return'] = (annual_data['Annual Profit'] / self.cash_required)/(year+1)

            data.append(annual_data)

        return data

    def to_dataframe(self):
        data = self.calculate_annual_data()
        columns = data[0].keys()
        df = pd.DataFrame(data, columns=columns)
        # df['Yearly Profit'] = df['Cummulative Profit'].diff()
        df.style.set_table_styles([dict(selector="th",props=[('max-width', '50px')])])
        # df = df.col.apply(lambda x: round(x, N - int(floor(log10(abs(x))))))
        # df = df.round(2)
        df = df.applymap(self.format_with_sig_figs)
        return df
    
    def applicable_months_per_year(self, total_months, total_years):
        rental_months = []
        for year in range(1, total_years + 1):
            if total_months >= 12:
                rental_months.append(12)
                total_months -= 12
            elif total_months > 0:
                rental_months.append(total_months)
                total_months = 0
            else:
                rental_months.append(0)
        return np.array(rental_months)
    
    def format_with_sig_figs(self, x):
        if x == 0:
            return "0"
        else:
            return "{:,.2f}".format(round(x, 2 - int(np.floor(np.log10(abs(x))))))


In [91]:
year_sum = YearlySummary(acq, rehab, pre_refi, refi, 30)
df = year_sum.to_dataframe()
df

,Year,Month,renting months,Total Annual Income,Operating Expenses,Mortgage Payment,Total Annual Expenses,Total Annual Cashflow,Cash on Cash ROI,Property Value,Loan Balance,Equity,Equity Gain,Annual Profit,Annualized Total Return
0,0,0,6.00,"24,000.00","6,930.00","45,600.00","52,500.00","-28,500.00",-0.22,"725,000.00","584,000.00","141,000.00","141,000.00","112,000.00",0.88
1,1.00,12.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","584,000.00","216,000.00","75,500.00","60,500.00",0.24
2,2.00,24.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","583,000.00","217,000.00",535.00,"-14,500.00",-0.04
3,3.00,36.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","583,000.00","217,000.00",537.00,"-14,500.00",-0.03
4,4.00,48.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","582,000.00","218,000.00",540.00,"-14,500.00",-0.02
5,5.00,60.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","582,000.00","218,000.00",543.00,"-14,500.00",-0.02
6,6.00,72.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","581,000.00","219,000.00",546.00,"-14,500.00",-0.02
7,7.00,84.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","581,000.00","219,000.00",549.00,"-14,500.00",-0.01
8,8.00,96.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","580,000.00","220,000.00",552.00,"-14,500.00",-0.01
9,9.00,108.00,12.00,"48,000.00","13,900.00","49,100.00","63,000.00","-15,000.00",-0.12,"800,000.00","580,000.00","220,000.00",555.00,"-14,500.00",-0.01
